# Model Implementation for 3D Cell Tracking


In [1]:
from torch.utils.data import DataLoader, random_split
from torch.utils.data.sampler import WeightedRandomSampler
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch
import torch.nn as nn
import numpy as np
import random 
import matplotlib.pyplot as plt

from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Designed VGG Model from Exercise 9

We will use a VGG network to classify the synapse images. The input to the network will be a 2D image as provided by your dataloader. The output will be a vector of six floats, corresponding to the probability of the input to belong to the six classes.

Implement a VGG network with the following specificatons:

* the constructor takes the size of the 2D input image as height and width
* the network starts with a downsample path consisting of:
    * one convolutional layer, kernel size (3, 3), to create 12 `fmaps`
    * `nn.BatchNorm2d` over those feature maps
    * `nn.ReLU` activation function
    * `nn.Conv2d` layer, kernel size (3, 3), to create 12 `fmaps`
    * `nn.BatchNorm2d` over those feature maps
    * `nn.ReLU` activation function
    * `nn.MaxPool2d` with a `downsample_factor` of (2, 2) at each level
* followed by three more downsampling paths like the one above, every time doubling the number of `fmaps` (i.e., the second one will have 24, the third 48, and the fourth 96). Make sure to keep track of the `current_fmaps` each time!
* then two times:
    * `nn.Linear` layer with `out_features=4096`. Be careful withe in `in_features` of the first one, which will depend on the size of the previous output!
    * `nn.ReLU` activation function
    * `nn.DropOut`
* Finally, one more fully connected layer with
    * `nn.Linear` to the 6 classes
    * no activation function 

Original One (from https://blog.paperspace.com/vgg-from-scratch-pytorch/)
https://github.com/pytorch/vision/blob/6db1569c89094cf23f3bc41f79275c45e9fcb3f3/torchvision/models/vgg.py#L24

# Create and Load Artificial Data
Make a fake dataset to test on the VGG model while waiting for data.


In [68]:
#original data size = 512x712x34 

fd_class1 = np.random.randint(low=30,high=60,size=(128,128, 128),dtype='int')
fd_class2 = np.random.randint(low=0,high=29,size=(128,128, 128),dtype='int')
y1 = 1
y2 = 0
fd_class1 = np.expand_dims(fd_class1, axis=0)
fd_class2 = np.expand_dims(fd_class2, axis=0)
loader = [(fd_class1, y1), (fd_class2,y2)]
#train = np.ndarray.flatten(fd_class1)
#train2 = np.ndarray.flatten(fd_class2)

# train_dataset, validation_dataset, test_dataset = random_split(
#     full_dataset,
#     [num_training, num_validation, num_test],
#     generator=torch.Generator().manual_seed(23061912))

#dataloader = DataLoader(train_dataset, batch_size=8, drop_last=True, sampler=sampler)


In [119]:
loader[0][1]

1

In [69]:
print(fd_class1.shape)
test = np.expand_dims(fd_class1, axis=0)
test.shape

(1, 128, 128, 128)


(1, 1, 128, 128, 128)

# My Attempt

In [125]:
class Vgg3D(torch.nn.Module):

    def __init__(self, input_size, output_classes, downsample_factors, fmaps=12):

        super(Vgg3D, self).__init__()

        self.input_size = input_size
        self.downsample_factors = downsample_factors
        self.output_classes = 2

        current_fmaps, h, w, d = tuple(input_size)
        current_size = (h, w,d)

        features = []
        for i in range(len(downsample_factors)):

            features += [
                torch.nn.Conv3d(current_fmaps,fmaps,kernel_size=3,padding=1),
                torch.nn.BatchNorm3d(fmaps),
                torch.nn.ReLU(inplace=True),
                torch.nn.Conv3d(fmaps,fmaps,kernel_size=3,padding=1),
                torch.nn.BatchNorm3d(fmaps),
                torch.nn.ReLU(inplace=True),
                torch.nn.MaxPool3d(downsample_factors[i])
            ]

            current_fmaps = fmaps
            fmaps *= 2

            size = tuple(
                int(c/d)
                for c, d in zip(current_size, downsample_factors[i]))
            check = (
                s*d == c
                for s, d, c in zip(size, downsample_factors[i], current_size))
            assert all(check), \
                "Can not downsample %s by chosen downsample factor" % \
                (current_size,)
            current_size = size

        self.features = torch.nn.Sequential(*features)

        classifier = [
            torch.nn.Linear(current_size[0] *current_size[1]*current_size[2] *current_fmaps,4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),
            torch.nn.Linear(4096,4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),
            torch.nn.Linear(4096,output_classes)
        ]

        self.classifier = torch.nn.Sequential(*classifier)
    
    def forward(self, raw):

        # add a channel dimension to raw
        # shape = tuple(raw.shape)
        # raw = raw.reshape(shape[0], 1, shape[1], shape[2])
        
        # compute features
        f = self.features(raw)
        f = f.view(f.size(0), -1)
        
        # classify
        y = self.classifier(f)

        return y

In [126]:
# get the size of our images
# for x, y in train_dataset:
#     input_size = x.shape
#     break
input_size = (1, 128, 128, 128)
downsample_factors =[(2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2)];
output_classes = 2
# create the model to train
model = Vgg3D(input_size, output_classes,  downsample_factors = downsample_factors)

In [127]:
loss = torch.nn.BCELoss()
#loss = torch.nn.CosineSimilarity()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [128]:
# use a GPU, if it is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"Will use device {device} for training")

Will use device cuda for training


# Training and Evaluation

In [93]:
!pip install torchsummary 

  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


In [129]:
from torchvision import models
from torchsummary import summary

summary( model, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 12, 128, 128, 128]             336
       BatchNorm3d-2    [-1, 12, 128, 128, 128]              24
              ReLU-3    [-1, 12, 128, 128, 128]               0
            Conv3d-4    [-1, 12, 128, 128, 128]           3,900
       BatchNorm3d-5    [-1, 12, 128, 128, 128]              24
              ReLU-6    [-1, 12, 128, 128, 128]               0
         MaxPool3d-7       [-1, 12, 64, 64, 64]               0
            Conv3d-8       [-1, 24, 64, 64, 64]           7,800
       BatchNorm3d-9       [-1, 24, 64, 64, 64]              48
             ReLU-10       [-1, 24, 64, 64, 64]               0
           Conv3d-11       [-1, 24, 64, 64, 64]          15,576
      BatchNorm3d-12       [-1, 24, 64, 64, 64]              48
             ReLU-13       [-1, 24, 64, 64, 64]               0
        MaxPool3d-14       [-1, 24, 32,

In [122]:
for x,y in loader:
    print(x.shape)

(1, 128, 128, 128)
(1, 128, 128, 128)


In [132]:
#Training on fake data
def train(model, loss_function):
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    model.train()
    epoch_loss = 0
    num_batches = 0
    model = model.to(device)
    
    for batch_id, (x, y) in enumerate(loader):
        # move input and target to the active device (either cpu or gpu)
        # x = np.stack(x, axis= 0)
        #x, y = x.to(device), y.to(device)
        x = x[np.newaxis]
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        print(x.shape)
        print(y.dtype)
        x, y = x.to(device), y.to(device)
        # zero the gradients for this iteration
        optimizer.zero_grad()

        # apply model and calculate loss
        prediction = model(x)
        loss = loss_function(prediction, y)

        # backpropagate the loss and adjust the parameters
        loss.backward()
        optimizer.step()

        # # log to console
        # if batch_id % log_interval == 0:
        #     print(
        #         "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
        #             epoch,
        #             batch_id * len(x),
        #             len(loader.dataset),
        #             100.0 * batch_id / len(loader),
        #             loss.item(),
        #         )
        #     )
    


In [134]:
for epoch in range(3):
    
    epoch_loss = train(model, loss)
    print(f"epoch {epoch}, training loss={epoch_loss}")
    
    # accuracy = validate()
    # print(f"epoch {epoch}, validation accuracy={accuracy}")

torch.Size([1, 1, 128, 128, 128])
torch.float32


ValueError: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 2])) is deprecated. Please ensure they have the same size.

In [32]:
for batch_id, (x,y) in enumerate(loader):
    print('x:', x)
    print('y: ', y)
    print('bat:', batch_id)

x: [[[34 48 39 ... 35 35 51]
  [58 58 31 ... 31 58 46]
  [55 43 59 ... 34 50 36]
  ...
  [47 32 47 ... 33 30 50]
  [44 57 31 ... 53 44 31]
  [33 42 48 ... 33 44 39]]

 [[55 31 50 ... 42 36 41]
  [35 58 56 ... 33 46 49]
  [56 33 44 ... 49 36 51]
  ...
  [35 36 51 ... 56 53 33]
  [37 40 50 ... 59 42 31]
  [44 40 45 ... 37 45 34]]

 [[41 45 54 ... 34 37 46]
  [51 40 58 ... 44 54 35]
  [36 50 46 ... 39 40 45]
  ...
  [46 36 48 ... 40 45 58]
  [31 40 49 ... 37 44 48]
  [43 40 54 ... 58 44 57]]

 ...

 [[37 59 52 ... 49 36 30]
  [58 49 49 ... 48 56 56]
  [54 40 58 ... 53 38 37]
  ...
  [37 33 31 ... 46 44 44]
  [39 45 50 ... 33 55 43]
  [41 47 39 ... 57 59 32]]

 [[35 33 50 ... 41 34 50]
  [30 43 46 ... 55 44 41]
  [58 37 57 ... 34 43 52]
  ...
  [59 40 37 ... 41 36 35]
  [31 33 38 ... 51 47 47]
  [44 58 42 ... 54 44 42]]

 [[50 33 42 ... 39 42 33]
  [56 40 56 ... 48 48 41]
  [36 46 57 ... 35 58 42]
  ...
  [36 34 43 ... 46 33 57]
  [34 45 57 ... 31 52 45]
  [33 42 36 ... 51 53 54]]]
y:  1
b

In [13]:
# from tqdm import tqdm

# def train():
#     '''Train the model for one epoch.'''

#     # set the model into train mode
#     model.train()

#     epoch_loss = 0

#     num_batches = 0
#     for x, y in tqdm(dataloader, 'train'):

#         x, y = x.to(device), y.to(device)
#         optimizer.zero_grad()

#         y_pred = model(x)
#         l = loss(y_pred, y)
#         l.backward()

#         optimizer.step()

#         epoch_loss += l
#         num_batches += 1

#     return epoch_loss/num_batches

# def evaluate(dataloader, name):
    
#     correct = 0
#     total = 0
#     for x, y in tqdm(dataloader, name):
        
#         x, y = x.to(device), y.to(device)
        
#         logits = model(x)
#         probs = torch.nn.Softmax(dim=1)(logits)
#         predictions = torch.argmax(probs, dim=1)
        
#         correct += int(torch.sum(predictions == y).cpu().detach().numpy())
#         total += len(y)
    
#     accuracy = correct/total

#     return accuracy

# def validate():
#     '''Evaluate prediction accuracy on the validation dataset.'''
    
#     model.eval()
#     dataloader = DataLoader(validation_dataset, batch_size=32)
   
#     return evaluate(dataloader, 'validate')

# def test():
#     '''Evaluate prediction accuracy on the test dataset.'''
    
#     model.eval()
#     dataloader = DataLoader(test_dataset, batch_size=32)
    
#     return evaluate(dataloader, 'test')

In [14]:
# for epoch in range(3):
    
#     epoch_loss = train()
#     print(f"epoch {epoch}, training loss={epoch_loss}")
    
#     accuracy = validate()
#     print(f"epoch {epoch}, validation accuracy={accuracy}")